![DataCamp courses](http://community.datacamp.com.s3.amazonaws.com/community/production/ckeditor_assets/pictures/293/content_blog_banner.png)

In [2]:
!pip install yfinance

yfinance lib: https://github.com/ranaroussi/yfinance/tree/main

In [92]:
import pandas as pd
import numpy as np
from typing import Tuple
from datetime import datetime, timezone
import matplotlib.pyplot as plt
import yfinance as yf

In [143]:
def get_historical_daily_market_data(symbols: str, start: datetime = None, end: datetime = None) -> Tuple[pd.DataFrame, datetime]:

  result = pd.DataFrame(columns=["symbol", "variable",	"value",	"actual_time"])
  max_ts: datetime = None
  symbols_list = symbols.split()
  unique_symbol = (True if len(symbols_list) == 1 else False)

  market_data = yf.download(symbols, start=start, end=end, interval="1d")
  if not market_data.empty:
    for symbol in symbols_list:

      dfs = pd.DataFrame(index=market_data.index)
      for col in list(market_data.columns):
        column = col if type(col).__name__ == "str" else col[0]
        dfs[column] = market_data[column] if unique_symbol else market_data[column][symbol]
      dfs["symbol"] = symbol
      dfs.reset_index(inplace=True)
      dfs = pd.melt(dfs, id_vars=['symbol', 'Date'])
      dfs["variable"] = dfs["variable"].str.lower().str.replace(" ", "_")
      dfs["symbol"] = dfs["symbol"].str.lower()
      dfs.columns = dfs.columns.str.lower()
      dfs["actual_time"] = dfs["date"]
      dfs.pop("date")

      result = pd.concat([result, dfs]).sort_values(by=["actual_time", "symbol"])
      max_ts: datetime = result.groupby("symbol", as_index=False).max().iloc[0]["actual_time"].to_pydatetime().astimezone(timezone.utc)

  return result, max_ts




In [144]:
symbols_as_str = "GOOG PLTR"
df, max_ts = get_historical_daily_market_data(symbols_as_str)

[*********************100%%**********************]  2 of 2 completed


In [145]:
df.head()

,symbol,variable,value,actual_time
0,goog,adj_close,2.499133,2004-08-19
4827,goog,close,2.499133,2004-08-19
9654,goog,high,2.591785,2004-08-19
14481,goog,low,2.390042,2004-08-19
19308,goog,open,2.490664,2004-08-19


In [146]:
max_ts

datetime.datetime(2023, 10, 20, 0, 0, tzinfo=datetime.timezone.utc)

In [105]:
import tempfile
tempfile.gettempdir()

'/tmp'

In [131]:
SOLUTION_NAME="tgedr_fetcher"
MAXTS_FILE_NAME=f"{SOLUTION_NAME}_maxts"
MAXTS_FILEPATH = f"{tempfile.gettempdir()}/.{MAXTS_FILE_NAME}"

def set_maxts(maxts: datetime):
  with open(MAXTS_FILEPATH, 'w') as f:
    f.write(str(int(maxts.timestamp())))

def get_maxts() -> datetime:
  result = None
  try:
    with open(MAXTS_FILEPATH) as f:
      for line in f:
          result = datetime.fromtimestamp(int(line), tz=timezone.utc)
          break
  except FileNotFoundError:
    print("file not found, shall we start from scratch?")
  return result




In [134]:
set_maxts(datetime.now().astimezone(timezone.utc))

In [137]:
get_maxts()

datetime.datetime(2023, 10, 22, 17, 12, 27, tzinfo=datetime.timezone.utc)

In [147]:
df, max_ts = get_historical_daily_market_data(symbols_as_str, start=get_maxts())

[*********************100%%**********************]  2 of 2 completed

ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['PLTR', 'GOOG']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-10-22 17:12:27+00:00 -> 2023-10-22)')
